In [1]:
import json
import re
import os
import pandas as pd
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px

pd.set_option('display.max_rows', 500)

In [49]:
file_path = r'..\data\Monthly Count Records (excel versions)\SEMBC_XLS_Yearly\embc2022.xlsx'
df = pd.read_excel(file_path,  sheet_name='May', header=None)

In [50]:
df[0] = df[0].fillna('')
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,"Date: May 8, 2022 Start: 07:15 Finish: 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tide State: low Tide Movement: falling ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Observers: Judith Vetsch, Patrick MacNamara, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Brian Storey, Christopher Di Corrado, Amenda N...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Species: 92,Total,OE,WD,SR,TD1,TD2,TD3,EF1,EF2,...,EM,BP,NF1,NF2,SA,IM,MC1,MC2,DW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,House Finch,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3,NaN,NaN,NaN,NaN,1,NaN,2,NaN
97,Pine Siskin,43,NaN,NaN,NaN,NaN,NaN,1,1,1,...,NaN,35,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN
98,American Goldfinch,12,NaN,NaN,NaN,NaN,NaN,NaN,1,1,...,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
99,,Total,OE,WD,SR,TD1,TD2,TD3,EF1,EF2,...,EM,BP,NF1,NF2,SA,IM,MC1,MC2,DW,NaN


In [6]:
info_1 = df.iloc[0,0]
info_2 = df.iloc[1,0]
info_3 = df.iloc[2,0]
info_4 = df.iloc[3,0]
info = info_1 + '  ' + info_2 + '  ' + info_3 + '  ' + info_4 + '  '


In [7]:
observers_match = re.search(r'(Observers: +)([\w -,]+\w+)(   *)', info)
observers = observers_match.group(2)
observers

'Dave Lassmann, Judith Vetsch, Patrick MacNamara, Chris Murrell, Grant Danielson, Sophie Vielfaure, Karl Ricker, Tiffany Brunke, Kyle Kulas, Chris Dale'

In [8]:
equipment_match = re.search(r'(Equipment: +)([\w -]+\w+)(   *)', info)
equipment = equipment_match.group(2)
equipment

'scope - binoculars only'

In [9]:
tide_movement_match = re.search(r'(Tide Movement: +)([\w -]+\w+)(   *)', info)
tide_movement = tide_movement_match.group(2)
tide_movement

'falling'

In [10]:
tide_state_match = re.search(r'(Tide State: +)([\w -]+\w+)(   *)', info)
tide_state = tide_state_match.group(2)
tide_state

'high'

In [11]:
sky_match = re.search(r'(Sky: +)([\w -]+\w+)(   *)', info)
sky = sky_match.group(2)
sky

'clear'

In [12]:
sea_state_match = re.search(r'(Sea State: +)([\w -]+\w+)(   *)', info)
sea_state = sea_state_match.group(2)
sea_state

'calm - ripple - chop'

In [13]:
date_match = re.search(r'(Date:\s+)([\w -,]+\w+)(   *)', info)
date = date_match.group(2)
date

'January 9, 2022'

In [14]:
start_match = re.search(r'(Start:\s+)(\w+:\w+)(   *)', info_1)
start = start_match.group(2)
start

'08:30'

In [15]:
finish_match = re.search(r'(Finish:\s+)(\w+:\w+)(   *)', info_1)
finish = finish_match.group(2)
finish

'2:00'

In [16]:
precip_match = re.search(r'(Precip: +)([\w -]+\w+)(   *)', info)
precip = precip_match.group(2)
precip

'none'

In [40]:
# get the header row
idx = df[df[0].str.contains(r'Species: +\w+', regex=True, na=False)].index
idx[0]

4

In [73]:
df1 = pd.read_excel(file_path,  sheet_name='Jul', header=idx[0])
columns = df1.columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       76 non-null     object
 1   1       75 non-null     object
 2   2       6 non-null      object
 3   3       10 non-null     object
 4   4       17 non-null     object
 5   5       11 non-null     object
 6   6       19 non-null     object
 7   7       14 non-null     object
 8   8       29 non-null     object
 9   9       24 non-null     object
 10  10      29 non-null     object
 11  11      24 non-null     object
 12  12      8 non-null      object
 13  13      29 non-null     object
 14  14      40 non-null     object
 15  15      17 non-null     object
 16  16      19 non-null     object
 17  17      20 non-null     object
 18  18      15 non-null     object
 19  19      34 non-null     object
 20  20      16 non-null     object
 21  21      22 non-null     object
dtypes: object(22)
memory usage: 

In [74]:

df2 = df1.dropna(subset=[columns[0]])
df2 = df2.drop(columns='Total')
df2 = df2.fillna(0)
df2 = df2.sort_values([columns[0]])
df2 = df2.set_index(columns[0])
df2.index.name=None
df2 = df2.drop('Totals')


In [46]:
# transpose to long form
df4 = pd.DataFrame()
for species in df2.index:
    species_count = df2.loc[species].T
    species_df = pd.DataFrame({'id': species_count.index, 'count': species_count.values, 'date': '2022-01', 'species': species})
    df4 = pd.concat([df4, species_df])
df4

,id,count,date,species
0,OE,0,2022-01,Accipiter sp.
1,WD,0,2022-01,Accipiter sp.
2,SR,0,2022-01,Accipiter sp.
3,TD1,0,2022-01,Accipiter sp.
4,TD2,0,2022-01,Accipiter sp.
...,...,...,...,...
15,SA,1,2022-01,Winter Wren (Pacific)
16,IM,0,2022-01,Winter Wren (Pacific)
17,MC1,0,2022-01,Winter Wren (Pacific)
18,MC2,0,2022-01,Winter Wren (Pacific)


In [41]:
# Total species
total_count = df2.gt(0).sum(axis=0)
df3 = pd.DataFrame({'id': total_count.index, 'counts': total_count.values, 'date':'2022-01', 'species': 'Total Species'})
df3

,id,counts,date,species
0,OE,6,2022-01,Total Species
1,WD,4,2022-01,Total Species
2,SR,7,2022-01,Total Species
3,TD1,5,2022-01,Total Species
4,TD2,8,2022-01,Total Species
5,TD3,3,2022-01,Total Species
6,EF1,11,2022-01,Total Species
7,EF2,5,2022-01,Total Species
8,CC,8,2022-01,Total Species
9,CS,12,2022-01,Total Species


# Load Data from excel sheets

In [68]:
months = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec'
]

excel_df = pd.DataFrame()
for year in range(2010,2023):
    file_path = os.path.join(r'..\data\Monthly Count Records (excel versions)\SEMBC_XLS_Yearly', f'embc{year}.xlsx')

    for month in months:
        df = pd.read_excel(file_path,  sheet_name=month, header=None)
        # idx = df[df[0].str.contains(r'Species: +\w+', regex=True, na=False)].index
        #find header row
        header_idx = df[df[2].str.contains(r'OE', regex=True, na=False)].index
        df1 = pd.read_excel(file_path,  sheet_name=month, header=header_idx[0])
        columns = df1.columns
        #find extra header rows and remove
        df1[columns[2]] = df1[columns[2]].astype('str')
        extra_header_idx = df1[df1[columns[2]].str.contains(r'OE', regex=True, na=False)].index
        if len(extra_header_idx) > 0:
            for header_row_idx in extra_header_idx:
                df1 = df1.drop(header_row_idx)
        
        df2 = df1.dropna(subset=[columns[0]])
        df2 = df2.drop(columns='Total')
        df2 = df2.fillna(0)
        df2 = df2.sort_values([columns[0]])
        total_idx = df2[df2[columns[0]].str.contains(r'Totals|Overall.*', regex=True, na=False)].index
        # df2.loc[total_idx,columns[0]] = 'Total Bird Count'
        df2 = df2.drop(total_idx)
        df2 = df2.set_index(columns[0])
        df2.index.name=None

        # individual species count
        for species in df2.index:
            species_count = df2.loc[species].T
            species_count = pd.to_numeric(species_count, errors='coerce').fillna(0)
            species_df = pd.DataFrame({'id': species_count.index, 'count': species_count.values, 'date':f'{month}-{year}', 'year': year, 'month': month,'species': species})
            excel_df = pd.concat([excel_df, species_df])

    
excel_df['count'] = excel_df['count'].astype('int64')

In [69]:
# Replacements Dictionary to correct spelling from excel sheets
replacements = {
    'Accipter sp.': 'Accipiter sp.',
    'American Crow x Northwestern Crow': 'American Crow',
    'American Robbin': 'American Robin',
    'American Wigeion': 'American Wigeon',
    'Anna’s Humming Bird': "Anna's Hummingbird",
    'Anna’s Hummingbird': "Anna's Hummingbird",
    'Back Swift': 'Black Swift',
    'Barrows Goldeneye': "Barrow's Goldeneye",
    "Barrrow's Goldeneye": "Barrow's Goldeneye",
    'Black-throated Grey Warbler': 'Black-throated Gray Warbler',
    'Blue Grouse (Sooty)': 'Sooty Grouse',
    'Bustit': 'Bushtit',
    'California (Western) Scrub-Jay': 'California Scrub-Jay',
    'Cedar Wagwing': 'Cedar Waxwing',
    'Chestnut-back Chickadee': 'Chestnut-backed Chickadee',
    'Doubel-crested Cormorant': 'Double-crested Cormorant',
    'Doule-crested Cormorant': 'Double-crested Cormorant',
    'Eurasian Collar-Dove': 'Eurasian Collared Dove',
    'Eurasian Collared-Dove': 'Eurasian Collared Dove',
    'Flycatcher, sp.': 'Flycatcher sp.',
    'Glaucous-winged x Herring Gull': 'Glaucous-winged Gull',
    'Gul sp.': 'Gull sp.',
    'Gull Hybrid': 'Gull, Hybrid',
    'Gull, Glaucous-winged x Western': 'Glaucous-winged Gull',
    'Gull, Western x Glaucous-wing': 'Glaucous-winged Gull',
    'Gull, Western x Glaucous-winged': 'Glaucous-winged Gull',
    'Gull,Glaucous-winged x Western': 'Glaucous-winged Gull',
    'Hawk Species': 'Hawk sp.',
    'Hawk, sp.': 'Hawk sp.',
    "Lincoln's Sparow": "Lincoln's Sparrow",
    'Lincoln’s Sparrow': "Lincoln's Sparrow",
    'Marsh Wren ': 'Marsh Wren',
    'Norther Harrier': 'Northern Harrier',
    'Northern Pygmy-owl': 'Northern Pygmy Owl',
    'Northern Pygmy-Owl': 'Northern Pygmy Owl',
    'Northwestern Crow': 'American Crow',
    'Nthn Rough-winged Swallow': 'Northern Rough-winged Swallow',
    'Nthn. Rough-winged Swallow': 'Northern Rough-winged Swallow',
    'Nthn. Rough-winges Swallow': 'Northern Rough-winged Swallow',
    'Olive-sided Flcatcher': 'Olive-sided Flycatcher',
    'Pacific-Slope Flycatcher': 'Pacific-slope Flycatcher',
    'Passerine sp': 'Passerine sp.',
    'Pelgagic Cormorant': 'Pelagic Cormorant',
    'Peregrin Falcon': 'Peregrine Falcon',
    'Peregrine': 'Peregrine Falcon',
    'Peregrine Falacon': 'Peregrine Falcon',
    'Pied -billed Grebe': 'Pied-billed Grebe',
    'Pied Bil Grebe': 'Pied-billed Grebe',
    'Pileated Wodpecker': 'Pileated Woodpecker',
    'Red Throated Loon': 'Red-throated Loon',
    'Red-tailed hawk': 'Red-tailed Hawk',
    'Ring-biled Gull': 'Ring-billed Gull',
    'Rock Dove': 'Pigeon',
    'Rock Dove (Pigeon)': 'Pigeon',
    'Rock Dove (Rock Pigeon)': 'Pigeon',
    'Rock Pigeon': 'Pigeon',
    'Ruby-crowned Kinglety': 'Ruby-crowned Kinglet',
    'Savannah': 'Savannah Sparrow',
    'Scaup Sp.': 'Scaup sp.',
    'Scaup sp': 'Scaup sp.',
    'Semi-palmated Sandpiper':  'Semipalmated Sandpiper',
    'Sharp-Shinned Hawk': 'Sharp-shinned Hawk', 
    'Sharp-shinned Hawk*': 'Sharp-shinned Hawk',
    'Short-blled Gull': 'Short-billed Gull',
    'Sooty (Blue) Grouse': 'Sooty Grouse',
    'Spotted Sandiper': 'Spotted Sandpiper',
    "Stellar's Jay": "Steller's Jay",
    'Steller’s Jay': "Steller's Jay",
    'Stellar’s Jay': "Steller's Jay",
    'Swainson’s Thrush': "Swainson's Thrush",
    'Townsend’s Warbler': "Townsend's Warbler",
    "Vaux' Swift": "Vaux's Swift",
    'Vaux’s Swift': "Vaux's Swift",
    'White-Winged Crossbill': 'White-winged Crossbill',
    'Winter (Pacific) Wren': 'Pacific Wren',
    'Winter Wren': 'Pacific Wren',
    'Winter Wren (Pacific Wren)': 'Pacific Wren',
    'Winter Wren (Pacific)': 'Pacific Wren',
    "Wison's Warbler": "Wilson's Warbler",
    'Woodpecker sp': 'Woodpecker sp.',
    'chickadee sp.': 'Chickadee sp.',
    'finch sp.': 'Finch sp.',
    'gull sp.': 'Gull sp.',
    'hummingbird sp.': 'Hummingbird sp.',
    'sparrow sp.': 'Sparrow sp.',
    'swallow sp.': 'Swallow sp.',
}

In [70]:
spell_checked_df = excel_df.replace(replacements)

In [71]:
# area total bird count 
areas_total_bird_count_df = spell_checked_df.groupby(['id','year','month', 'date']).sum('count').reset_index()
areas_total_bird_count_df['species'] = 'Total Bird Count'


areas_total_added_df = pd.concat([spell_checked_df, areas_total_bird_count_df])
all_area_total_bird = areas_total_added_df.groupby(['year','month', 'date', 'species']).sum('count').reset_index()
all_area_total_bird['id'] = 'ALL'


all_area_total_added_df = pd.concat([areas_total_added_df, all_area_total_bird])
all_area_total_added_df[all_area_total_added_df['species'] != 'Total Bird Count']



,id,count,date,year,month,species
0,OE,0,Jan-2010,2010,Jan,American Robin
1,WD,0,Jan-2010,2010,Jan,American Robin
2,SR,0,Jan-2010,2010,Jan,American Robin
3,TD1,0,Jan-2010,2010,Jan,American Robin
4,TD2,0,Jan-2010,2010,Jan,American Robin
...,...,...,...,...,...,...
9711,ALL,1,Sep-2022,2022,Sep,Violet-green Swallow
9712,ALL,1,Sep-2022,2022,Sep,Virginia Rail
9713,ALL,55,Sep-2022,2022,Sep,White-crowned Sparrow
9714,ALL,1,Sep-2022,2022,Sep,Willow Flycatcher


In [72]:

# sum species in each area, making sure to remove "Total Bird Count" from totals
all_area_species = all_area_total_added_df[all_area_total_added_df['species']!='Total Bird Count']
all_area_total_species = all_area_species.groupby(['id', 'date', 'year', 'month']).apply(lambda x: x['count'].gt(0).sum(axis=0)).reset_index(name='count')
all_area_total_species['species'] = 'Total Species Count'


areas_and_species_totals_added_df = pd.concat([all_area_total_added_df, all_area_total_species])
# final = pd.concat([final, all_area_total_bird])

In [73]:
# test_final3[test_final3['month'].isnull()]
len(areas_and_species_totals_added_df.species.unique())

252

In [74]:
df_loaded = areas_and_species_totals_added_df
df_loaded['month'] = pd.Categorical(df_loaded['month'], months)
df_loaded = df_loaded.sort_values(['year', 'month','species']).reset_index(drop=True)
# df_loaded

In [75]:
merge_test = df_loaded[(df_loaded['year']==2011) & (df_loaded['id']=='ALL')]
merge_test

,id,count,date,year,month,species
16083,ALL,1,Jan-2011,2011,Jan,Accipiter sp.
16104,ALL,34,Jan-2011,2011,Jan,American Crow
16125,ALL,4,Jan-2011,2011,Jan,American Robin
16146,ALL,28,Jan-2011,2011,Jan,American Wigeon
16167,ALL,5,Jan-2011,2011,Jan,Bald Eagle
...,...,...,...,...,...,...
31830,ALL,56,Dec-2011,2011,Dec,Total Species Count
31871,ALL,4,Dec-2011,2011,Dec,Trumpeter Swan
31892,ALL,8,Dec-2011,2011,Dec,Varied Thrush
31913,ALL,3,Dec-2011,2011,Dec,Western Meadowlark


In [76]:
months = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec']

year =2011
merge_template = pd.DataFrame()
for specie in merge_test.species.unique():
    temp = pd.DataFrame({'id': 'ALL', 'date': [f'{month}-{year}' for month in months], 'year': year, 'month':months, 'species': specie})
    merge_template = pd.concat([merge_template, temp])
# merge_template.merge(merge_test, how='left', on=['id', 'date', 'year', 'month', 'species']).fillna(0)
merge_df = merge_template.merge(merge_test, how='left', on=['id', 'date', 'year', 'month', 'species']).fillna(0)

In [77]:
df=pd.concat([df_load[(df_load['year']==2011) & (df_load['id']!='ALL')],merge_df])

In [78]:
df = df.reset_index(drop=True)

In [20]:
import json
import plotly.express as px
with open(r"C:\Users\kylek\OneDrive\Documents\Code\shared_with_VM\bird_count\data\areas.json") as file:
    areas = json.load(file)


fig = px.choropleth_mapbox(final, geojson=areas, locations='id', color='counts',
                           color_continuous_scale="Purp",
                           range_color=(0, final.counts.max()),
                           mapbox_style="carto-positron",
                           zoom=12.5, center = {"lat": 49.7, "lon": -123.15},
                           opacity=0.5,
                           labels={'counts':'Total Species', 'id': "Area"},
                           animation_frame="date"
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [112]:

# importing packages
import plotly.express as px
  
# fig = px.colors.sequential.swatches_continuous()
fig = px.colors.qualitative.swatches()
fig.show()

In [108]:
px.colors.qualitative

<module '_plotly_utils.colors.qualitative' from 'c:\\Users\\kylek\\OneDrive\\Documents\\Code\\shared_with_VM\\bird_count\\dev\\.dev_env\\lib\\site-packages\\_plotly_utils\\colors\\qualitative.py'>

read areas and mapbox token

In [79]:
with open(
    r"C:\Users\kylek\OneDrive\Documents\Code\shared_with_VM\bird_count\data\areas.json"
) as areas_file:
    areas = json.load(areas_file)
with open(
    r"C:\Users\kylek\OneDrive\Documents\Code\shared_with_VM\bird_count\.mapbox_token"
) as token_file:
    token = token_file.read()



Get Data from csv

In [49]:
df_load = pd.read_csv(r'C:\Users\kylek\OneDrive\Documents\Code\shared_with_VM\bird_count\bird_count\data\count_data.csv', index_col=0)

In [ ]:
df=df_loaded

# App

In [80]:
months = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec']

app = JupyterDash(__name__)

app.layout = html.Div(
    id="app-container",
    children=[
        html.Div(
            id="sidebar-container",
            children=[
                html.H1("Squamish Monthly Bird Count"),
                dcc.Tabs(
                    id="tabs",
                    value="tab-map",
                    children=[
                        dcc.Tab(label="Map", value="tab-map"),
                        dcc.Tab(label="Graph", value="tab-graph"),
                    ],
                ),
                html.Div([
                    html.P("Select Species:"),
                    dcc.Dropdown(
                        id="species-dropdown",
                        options=sorted(df["species"].unique()),
                        value="Total Species Count",
                        clearable=False,
                    ),
                ]),
            ],
        ),
        html.Div(
            id="content-container",
        ),
    ],
)


@app.callback(
    Output('content-container', 'children'),
    Input('tabs', 'value')
)
def render_content_container(tab):
    if tab == 'tab-map':
        return dcc.Graph(
            id="count-map",
            config=dict(responsive=True),
        )
    elif tab == 'tab-graph':
        return dcc.Graph(
            id="count-graph",
            config=dict(responsive=True),
        )

@app.callback(
    Output("count-map", "figure"), 
    Input("species-dropdown", "value")
)
def update_map(species):
    dff = df[df["species"] == species]
    
    fig = px.choropleth_mapbox(
        dff,
        geojson=areas,
        locations="id",
        featureidkey="properties.id",
        color="count",
        color_continuous_scale="Purples",
        range_color=(0, dff["count"].max()),
        zoom=12.5,
        center={"lat": 49.7, "lon": -123.15},
        opacity=0.5,
        labels={"count": "Count", "id": "Area"},
        animation_frame="date",
        template="plotly_dark",
    )
    fig.update_layout(
        margin={"r": 20, "t": 20, "l": 20, "b": 20},
        mapbox_accesstoken=token,
        mapbox_style="satellite-streets",
    )
    if fig["layout"]["updatemenus"]:
        fig["layout"]["updatemenus"][0]["pad"] = dict(r=20, t=25)
        fig["layout"]["sliders"][0]["pad"] = dict(r=0, t=0, b=20)

    return fig

@app.callback(
    Output("count-graph", "figure"), 
    Input("species-dropdown", "value")
)
def update_graph(species):
    # dff = df[df['species'] == species].groupby(['species','year','month']).sum('count').reset_index()
    dff = df[(df['species']== species) & (df['id']=='ALL')]

    fig = px.line(dff,
        x='month',
        y='count',
        category_orders= {'month': months},
        color="year",
        color_discrete_sequence=px.colors.qualitative.Light24,
        # color_discrete_sequence=px.colors.qualitative.Alphabet,
        template="plotly_dark",
        markers=True
    )

    fig.update_layout(
        margin={"r": 20, "t": 20, "l": 20, "b": 20},
    )

    return fig


if __name__ == "__main__":
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
